In [9]:
import json
prompt = "/home/xinyuan/workspace/llm-reasoners/examples/AQuA_rap/prompts/score_examples.json"
with open(prompt) as f:
    prompt = json.load(f)

In [10]:
prompt

[{'input': "Given a question and its sub-questions and sub-answers, determine whether the new sub-answer is logically correct. Output 'Yes' or 'No', and a reason.\n\nQuestion 1: When 20 percent of a number is added to another number, the second number increases to its 140 per cent. What is the ratio between the first and the second number? Options: A)3 : 4, B)2 : 1, C)3 : 2, D)Data inadequate, E)None of these\nQuestion 1.1: Let the first number be x and the second number be y. What equation satisfied the increase?\nNew answer 1.1: the equation is y + 0.2 x = 1.4y. The answer is y + 0.2x = 1.4y.\nIs the new answer logically correct? Yes. The sub-answer follows the sub-question, and the equation shows the correct relationship between x and y.\n\nQuestion 2: In a certain store, the profit is 320% of the cost. If the cost increases by 25% but the selling price remains constant, approximately what percentage of the selling price is the profit? Options: A)80%, B)70%, C)60%, D)50%, E)40%\nQue

In [2]:
from inference import eval_non_aggregate
eval_non_aggregate(pkl_pth="/home/xinyuan/workspace/llm-reasoners/logs/AQuA_clean_MCTS/11292023-190437/algo_output", resume_s=0, resume_e=206)

match: D
match: E
match: A
match: D
match: C
match: B
match: A
match: C
match: D
match: D
match: D
match: A
match: C
match: C
match: A
match: D
match: D
match: D
match: E
match: E
match: A
match: D
match: C
match: C
match: D
match: B
match: C
match: E
match: C
match: B
match: C
match: C
match: B
match: C
match: B
match: E
match: C
match: C
match: E
match: B
match: A
match: B
match: D
match: B
match: B
match: A
match: E
match: A
match: B
match: A
match: D
match: D
match: D
match: A
match: E
match: A
match: D
match: A
match: C
match: A
match: A
match: C
match: E
match: B
match: C
match: C
match: C
match: A
match: D
match: C
match: A
match: A
match: E
match: A
match: B
match: A
match: B
match: E
match: B
match: D
match: B
match: B
match: B
match: B
match: A
match: B
match: A
match: C
match: C
match: E
match: D
match: C
match: B
match: C
match: A
match: D
match: C
match: B
match: C
match: C
match: B
match: B
match: C
match: A
match: D
match: C
match: D
match: D
match: A
match: E
match: D
m

In [12]:
import re
def retrieve_answer(output):
    '''
    output should be a world_model.AMTHState if being a list
    '''
    if isinstance(output, list):
        output = output[-1].sub_answer
    match = re.match(r'.*[Tt]he answer is.*?([A-E]).*?$', output, re.DOTALL)
    print(match)
    if match is None:
        return None
    answer = match[1].strip()
    
    return answer
output = "fiauef the answer is A. fabwiehkf"
retrieve_answer(output)

<re.Match object; span=(0, 33), match='fiauef the answer is A. fabwiehkf'>


'A'

In [1]:
import pickle
from typing import Type, Callable, Optional

import numpy as np
from datasets import load_dataset
from tqdm import tqdm
from datetime import datetime

from reasoners import LanguageModel, Reasoner, SearchAlgorithm
from reasoners.algorithm import MCTS

from world_model import MATHWorldModel
from search_config import MATHConfig
import utils

from datasets import Dataset
import os
import re
import json

from inference import data_reader

/home/xinyuan/anaconda3/envs/reasoners/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
datasetname: str = 'AQuA_clean'
dataset_path: str = 'dataset/AQuA'
dataset = data_reader(datasetname, dataset_path)


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 908.25it/s]
Generating train split: 207 examples [00:00, 43602.90 examples/s]


In [6]:
dataset[0]

{'question': 'A car is being driven, in a straight line and at a uniform speed, towards the base of a vertical tower. The top of the tower is observed from the car and, in the process, it takes 10 minutes for the angle of elevation to change from 45° to 60°. After how much more time will this car reach the base of the tower? Options: A) 5(√3 + 1) B) 6(√3 + √2) C) 7(√3 – 1) D) 8(√3 – 2) E) None of these.',
 'answer': 'A',
 'options': ['A)5(√3 + 1)',
  'B)6(√3 + √2)',
  'C)7(√3 – 1)',
  'D)8(√3 – 2)',
  'E)None of these']}